# Intelligent Query Processing
## Table Variable Deferred Compilation
Table Variable Deferred Compilation is an example of Intelligent Query Processing that can be enabled with database compatibility level 150 or higher. Follow the steps in this notebook to see the performance difference when table variable compilation is deferred to provide a better query plan.

**Prerequisites**: Restore the WideWorldImportersDW full backup. Then run the **extendwwidw.sql** script to customize the database. If you have already done this, proceed to the next steps

## Step 1: Create a stored procedure with a table variable
Create a stored procedure that will calculate the top 10 orders in the system by price. Make it a fair comparison by warming the cache with the OrderHistoryExtended table.

In [1]:
-- Step 1: Create a stored procedure using a table variable to show the top 10 past orders by price
USE WideWorldImportersDW
GO
CREATE OR ALTER PROCEDURE Fact.OrderPrices
AS
BEGIN
DECLARE @Order TABLE 
	([Order Key] BIGINT NOT NULL,
	 [Quantity] INT NOT NULL
	)

INSERT @Order
SELECT [Order Key], [Quantity]
FROM [Fact].[OrderHistory]

-- Look at estimated rows, speed, join algorithm
SELECT top 10 oh.[Order Key], oh.[Order Date Key],oh.[Unit Price], o.Quantity
FROM Fact.OrderHistoryExtended AS oh
INNER JOIN @Order AS o
ON o.[Order Key] = oh.[Order Key]
WHERE oh.[Unit Price] > 0.10
ORDER BY oh.[Unit Price] DESC
END
GO

Commands completed successfully.

Total execution time: 00:00:00.0015615

Commands completed successfully.

Total execution time: 00:00:00.0272941

## Step 2: Change the database to legacy behavior
Change to dbcompat 130 and clear the procedure cache

In [2]:
-- Step 2: Change WideWorldImportersDW to dbcompat 130 to see legacy behavior
ALTER DATABASE SCOPED CONFIGURATION CLEAR PROCEDURE_CACHE
GO
ALTER DATABASE WideWorldImportersDW SET COMPATIBILITY_LEVEL = 130
GO
SELECT COUNT(*) FROM Fact.OrderHistoryExtended
GO

Commands completed successfully.

Total execution time: 00:00:00.0097917

Commands completed successfully.

Total execution time: 00:00:00.0136308

(1 row affected)

Total execution time: 00:00:00.8205541

(No column name)
29620736


## Step 3: Run the procedure a few times and check performance
A single execution may be ok but what happens if you run it mulitple times. Any degradation of performance just feels worse.

In [3]:
-- Step 3: Run the Fact.OrderPrices procedure 3 times
EXEC Fact.OrderPrices
GO 3

Beginning execution loop

(3702592 rows affected)

(10 rows affected)

(3702592 rows affected)

(10 rows affected)

...

Batch execution completed 3 times...

Total execution time: 00:01:28.8783119

Order Key,Order Date Key,Unit Price,Quantity
3109626,2016-02-04,1899.00,1
1560766,2015-10-07,1899.00,1
2483890,2014-07-28,1899.00,1
2434821,2014-05-21,1899.00,1
2776661,2015-09-22,1899.00,1
2395284,2013-12-21,1899.00,1
3389382,2013-11-12,1899.00,1
3616967,2015-05-05,1899.00,1
338621,2013-10-31,1899.00,1
541981,2015-09-21,1899.00,1


Order Key,Order Date Key,Unit Price,Quantity
2445238,2016-02-04,1899.00,1
1118368,2015-05-05,1899.00,1
3336508,2015-07-30,1899.00,1
9560,2013-03-04,1899.00,1
138019,2015-02-23,1899.00,1
692897,2014-05-07,1899.00,1
794205,2013-09-30,1899.00,1
1457172,2014-08-21,1899.00,1
3346961,2013-12-21,1899.00,1
318597,2015-07-30,1899.00,1


Order Key,Order Date Key,Unit Price,Quantity
2384839,2016-02-17,1899.00,1
1148161,2013-03-27,1899.00,1
85921,2014-05-16,1899.00,1
86819,2014-05-21,1899.00,1
1728777,2013-07-26,1899.00,1
1940365,2013-12-21,1899.00,1
2541198,2015-05-05,1899.00,1
2876730,2013-09-30,1899.00,1
2786228,2013-03-27,1899.00,1
3058258,2015-12-17,1899.00,1


## Step 4: Change the database to enable Intelligent Query Processing
Change to dbompcat 150 and clear the procedure cache to enable IQP and table variable deferred compilation

In [4]:
-- Step 4: Change WideWorldImportersDW to dbcompat 150 to see legacy behavior
ALTER DATABASE SCOPED CONFIGURATION CLEAR PROCEDURE_CACHE
GO
ALTER DATABASE WideWorldImportersDW SET COMPATIBILITY_LEVEL = 150
GO

Commands completed successfully.

Total execution time: 00:00:00.0046530

Commands completed successfully.

Total execution time: 00:00:00.0131408

## Step 5: Run the procedure again with IQP enabled
Let's see if the procedure is any faster

In [5]:
-- Step 5: Is it any faster?
EXEC Fact.OrderPrices
GO 3

Beginning execution loop

(3702592 rows affected)

(10 rows affected)

(3702592 rows affected)

(10 rows affected)

...

Batch execution completed 3 times...

Total execution time: 00:00:10.5549412

Order Key,Order Date Key,Unit Price,Quantity
2488307,2015-07-25,1899.00,1
2488053,2014-04-18,1899.00,1
2525602,2013-07-19,1899.00,4
2524183,2016-04-26,1899.00,10
2525500,2013-06-24,1899.00,5
2487977,2014-03-28,1899.00,7
2488497,2015-09-08,1899.00,3
2488245,2014-05-09,1899.00,10
2488128,2014-05-02,1899.00,4
2487354,2014-02-05,1899.00,9


Order Key,Order Date Key,Unit Price,Quantity
2420632,2014-05-10,1899.00,3
2418809,2015-11-03,1899.00,6
2467712,2013-06-07,1899.00,3
2420714,2014-05-29,1899.00,6
2421938,2014-07-31,1899.00,2
2420155,2015-05-26,1899.00,5
2420647,2014-05-15,1899.00,2
2419056,2015-12-19,1899.00,5
2419132,2015-05-29,1899.00,6
2418867,2014-05-20,1899.00,6


Order Key,Order Date Key,Unit Price,Quantity
2488307,2015-07-25,1899.00,1
2488053,2014-04-18,1899.00,1
2525602,2013-07-19,1899.00,4
2524183,2016-04-26,1899.00,10
2525500,2013-06-24,1899.00,5
2487977,2014-03-28,1899.00,7
2488497,2015-09-08,1899.00,3
2488245,2014-05-09,1899.00,10
2488128,2014-05-02,1899.00,4
2487354,2014-02-05,1899.00,9
